In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
import numpy as np
# concat to df
# 
# df and sims should have same # of rows
def form_most_similar(df, sims, n):
    # sorts smallest to biggest for the provided similarity matrix
    # this call takes about a minute for 24k rows
    sorted_inds = np.argsort(sims)
    
    # put a graph index in
    df["graphID"] = np.arange(len(df))
    # put similarity cols in
    for i in range(n):
        col_name = "top_sim_" + str(i)
        df[col_name] = -1
        col_name = "bot_sim_" + str(i)
        df[col_name] = -1
        
    # direction: top - get the top n, bot - get the bot n
    # assumes row is sorted smallest to biggest
    # will ignore row_i if it sees it
    def get_n(row, row_i, n, direction):
        collected_n = []
        row_j = 0
        if direction == "top":
            row_j = len(row) - 1
        added = 0
        while added < n and row_j >= 0 and row_j < len(row):
            if row[row_j] != row_i:
                collected_n.append(row[row_j])
                added = added + 1
            if direction == "top":
                row_j = row_j - 1
            else:
                row_j = row_j + 1
        return collected_n
            
    # form the top and bottom n similar tenders
    for i, row in enumerate(sorted_inds):
        bot_n = get_n(row, i, n, "bot")
        top_n = get_n(row, i, n, "top")
        for j in range(n):
            col_name = "top_sim_" + str(j)
            df.iloc[i,col_name] = top_n[j]
            col_name = "bot_sim_" + str(j)
            df.iloc[i,col_name] = bot_n[j]

In [31]:
import pandas as pd
fast_text_embs = pd.read_csv("../fasttext_emb/fasttext_emb.csv", header = None)
tenders = pd.read_excel("../../../data/UpdatedAgainTenders.xlsx")

In [35]:
sims = cosine_similarity(fast_text_embs)

In [68]:
%%time
form_most_similar(tenders, sims, 5)
tenders

KeyboardInterrupt: 